In [1]:
import sys
import time

sys.path.append("../")
from calculus.pseudonym import *
from calculus.strategy import *

# <span style="color: #A1F99C">TEST PROCEDURE</span>

In [2]:
rs = RightmostInnermostStrategy()


def test_procedure(term, reduction_strategy=rs, print_steps=True,
                   steps_lim=60, assert_result=""):
    term = term._updateBoundVariables()
    n_steps = 0
    is_term_normalized = True
    if print_steps:
        print(f"{n_steps}: {term}")
    while term.redexes:
        term = term._betaConversion(reduction_strategy)
        term = term._updateBoundVariables()
        n_steps += 1
        if print_steps:
            print(f"{n_steps}: {term}")
        if n_steps > steps_lim:
            is_term_normalized = False
            break

    print(f"\n\nreduction steps: {n_steps}")
    print(f"norm term:       {term}" if is_term_normalized else f"NOT norm term: {term}")

# <span style="color: #A1F99C">USEFUL TERMS</span>

In [3]:
def ite_term():
    x, y, c = Var(), Var(), Var()
    x_, y_, c_ = Atom(x), Atom(y), Atom(c)
    return Lambda(c, Lambda(x, Lambda(y, multi_app_term(c_, x_, y_))))


def true_term():
    x, y = Var(), Var()
    x_ = Atom(x)
    return Lambda(x, Lambda(y, x_))


def num_zero_term():
    s, z = Var(), Var()
    z_ = Atom(z)
    return Lambda(s, Lambda(z, z_))


def num_term(n: int):
    if n <= 0:
        return num_zero_term()
    s, z = Var(), Var()
    s_, z_ = Atom(s), Atom(z)
    core_term = App(s_, z_)
    for _ in range(n - 1):
        core_term = App(s_, core_term)
    return Lambda(s, Lambda(z, core_term))


def multi_app_term(term_0: Term, term_1: Term, *terms: Term):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term


def false_term():
    x, y = Var(), Var()
    y_ = Atom(y)
    return Lambda(x, Lambda(y, y_))


def pair_term():
    x, y, p = Var(), Var(), Var()
    x_, y_, p_ = Atom(x), Atom(y), Atom(p)
    return Lambda(x, Lambda(y, Lambda(p, multi_app_term(p_, x_, y_))))


def first_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, true_term()))


def second_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, false_term()))


def succ_term():
    x, y, n = Var(), Var(), Var()
    x_, y_, n_ = Atom(x), Atom(y), Atom(n)
    return Lambda(n, Lambda(x, Lambda(y, App(x_, multi_app_term(n_, x_, y_)))))


def sinc_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(
        p,
        multi_app_term(
            pair_term(),
            App(second_term(), p_),
            App(succ_term(), App(second_term(), p_)),
        ),
    )


def pred_term():
    n, f, x, g, h, u = Var(), Var(), Var(), Var(), Var(), Var()
    n_, f_, x_ = Atom(n), Atom(f), Atom(x)
    g_, h_, u_ = Atom(g), Atom(h), Atom(u)

    return Lambda(n, Lambda(f, Lambda(x, multi_app_term(
        n_,
        Lambda(g, Lambda(h, App(h_, App(g_, f_)))),
        Lambda(u, x_),
        Lambda(u, u_)
    ))))


def plus_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(m, Lambda(n, multi_app_term(n_, succ_term(), m_)))


def subtract_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(n, Lambda(m, multi_app_term(m_, pred_term(), n_)))


def iszero_term():
    x, n = Var(), Var()
    n_ = Atom(n)
    return Lambda(n, App(App(n_, Lambda(x, false_term())), true_term()))


def leq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n, Lambda(m, App(iszero_term(), multi_app_term(subtract_term(), n_, m_)))
    )


def and_term():
    a, b = Var(), Var()
    a_, b_ = Atom(a), Atom(b)
    return Lambda(a, Lambda(b, multi_app_term(ite_term(), a_, b_, a_)))


def eq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n,
        Lambda(
            m,
            multi_app_term(
                and_term(),
                multi_app_term(leq_term(), n_, m_),
                multi_app_term(leq_term(), m_, n_),
            ),
        ),
    )

# <span style="color: #A1F99C">1.</span>
<span style="color:#bb91f3">(λx. (λy.y)) ((λx. (xx)) (λx. (xx)))</span> == <span style="color:#f1bd47">I</span> == <span style="color:#f1bd47">λx. x</span>
<span style="color:#c9fafa">This term reduce in one step by LO, RO strategy
And reduce in the same terms with other strategy.</span>

In [4]:
def term_test_1():
    x, y = Var(), Var()
    x_, y_ = Atom(x), Atom(y)

    return App(
             Lambda(x, Lambda(y, y_)),
             App(
                 Lambda(x, App(x_, x_)),
                 Lambda(x, App(x_, x_))
             )
         )


test_procedure(term_test_1())

0: ((λq.(λw.w)) ((λe.(e e)) (λr.(r r))))
1: ((λe.(λr.r)) ((λq.(q q)) (λw.(w w))))
2: ((λq.(λw.w)) ((λe.(e e)) (λr.(r r))))
3: ((λe.(λr.r)) ((λq.(q q)) (λw.(w w))))
4: ((λq.(λw.w)) ((λe.(e e)) (λr.(r r))))
5: ((λe.(λr.r)) ((λq.(q q)) (λw.(w w))))
6: ((λq.(λw.w)) ((λe.(e e)) (λr.(r r))))
7: ((λe.(λr.r)) ((λq.(q q)) (λw.(w w))))
8: ((λq.(λw.w)) ((λe.(e e)) (λr.(r r))))
9: ((λe.(λr.r)) ((λq.(q q)) (λw.(w w))))
10: ((λq.(λw.w)) ((λe.(e e)) (λr.(r r))))
11: ((λe.(λr.r)) ((λq.(q q)) (λw.(w w))))
12: ((λq.(λw.w)) ((λe.(e e)) (λr.(r r))))
13: ((λe.(λr.r)) ((λq.(q q)) (λw.(w w))))
14: ((λq.(λw.w)) ((λe.(e e)) (λr.(r r))))
15: ((λe.(λr.r)) ((λq.(q q)) (λw.(w w))))
16: ((λq.(λw.w)) ((λe.(e e)) (λr.(r r))))
17: ((λe.(λr.r)) ((λq.(q q)) (λw.(w w))))
18: ((λq.(λw.w)) ((λe.(e e)) (λr.(r r))))
19: ((λe.(λr.r)) ((λq.(q q)) (λw.(w w))))
20: ((λq.(λw.w)) ((λe.(e e)) (λr.(r r))))
21: ((λe.(λr.r)) ((λq.(q q)) (λw.(w w))))
22: ((λq.(λw.w)) ((λe.(e e)) (λr.(r r))))
23: ((λe.(λr.r)) ((λq.(q q)) (λw.(w w))))
24

# <span style="color: #A1F99C">2.</span>
<span style="color:#bb91f3">(ITE TRUE 3 1)</span> == <span style="color:#f1bd47>">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [5]:
def term_test_2():
    return multi_app_term(ite_term(), true_term(), num_term(3), num_term(1))


test_procedure(term_test_2())

0: ((((λq.(λw.(λe.((q w) e)))) (λr.(λt.r))) (λy.(λu.(y (y (y u)))))) (λi.(λo.(i o))))
1: (((λq.(λw.(((λe.(λr.e)) q) w))) (λt.(λy.(t (t (t y)))))) (λu.(λi.(u i))))
2: (((λq.(λw.((λe.q) w))) (λr.(λt.(r (r (r t)))))) (λy.(λu.(y u))))
3: (((λw.(λe.w)) (λr.(λt.(r (r (r t)))))) (λy.(λq.(y q))))
4: ((λq.(λw.(λe.(w (w (w e)))))) (λr.(λt.(r t))))
5: (λq.(λw.(q (q (q w)))))


reduction steps: 5
norm term:       (λq.(λw.(q (q (q w)))))


# <span style="color: #A1F99C">3.</span>
<span style="color:#bb91f3">(ITE FALSE 0 4)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [6]:
def term_test_3():
    return multi_app_term(ite_term(), false_term(), num_term(0), num_term(4))


test_procedure(term_test_3())

0: ((((λq.(λw.(λe.((q w) e)))) (λr.(λt.t))) (λy.(λu.u))) (λi.(λo.(i (i (i (i o)))))))
1: (((λe.(λr.(((λt.(λy.y)) e) r))) (λu.(λi.i))) (λq.(λw.(q (q (q (q w)))))))
2: (((λq.(λw.((λe.e) w))) (λr.(λt.t))) (λy.(λu.(y (y (y (y u)))))))
3: (((λq.(λw.w)) (λe.(λr.r))) (λt.(λy.(t (t (t (t y)))))))
4: ((λe.e) (λq.(λw.(q (q (q (q w)))))))
5: (λq.(λw.(q (q (q (q w))))))


reduction steps: 5
norm term:       (λq.(λw.(q (q (q (q w))))))


# <span style="color: #A1F99C">4.</span>
<span style="color:#bb91f3">(PRED 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [7]:
def term_test_4():
    return multi_app_term(pred_term(), num_term(0))


test_procedure(term_test_4())

0: ((λy.(λu.(λi.(((y (λo.(λq.(q (o u))))) (λw.i)) (λe.e))))) (λr.(λt.t)))
1: (λq.(λw.((((λe.(λr.r)) (λt.(λy.(y (t q))))) (λu.w)) (λi.i))))
2: (λq.(λw.(((λe.e) (λr.w)) (λt.t))))
3: (λq.(λw.((λe.w) (λr.r))))
4: (λq.(λw.w))


reduction steps: 4
norm term:       (λq.(λw.w))


# <span style="color: #A1F99C">5.</span>
<span style="color:#bb91f3">(PRED 5)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [8]:
def term_test_5():
    return multi_app_term(pred_term(), num_term(5))


test_procedure(term_test_5())

0: ((λq.(λw.(λe.(((q (λr.(λt.(t (r w))))) (λy.e)) (λu.u))))) (λi.(λo.(i (i (i (i (i o))))))))
1: (λq.(λw.((((λe.(λr.(e (e (e (e (e r))))))) (λt.(λy.(y (t q))))) (λu.w)) (λi.i))))
2: (λq.(λw.(((λe.((λr.(λt.(t (r q)))) ((λy.(λu.(u (y q)))) ((λi.(λo.(o (i q)))) ((λp.(λa.(a (p q)))) ((λs.(λd.(d (s q)))) e)))))) (λf.w)) (λg.g))))
3: (λq.(λw.(((λe.((λr.(λt.(t (r q)))) ((λy.(λu.(u (y q)))) ((λi.(λo.(o (i q)))) ((λp.(λa.(a (p q)))) (λs.(s (e q)))))))) (λd.w)) (λf.f))))
4: (λq.(λw.(((λe.((λr.(λt.(t (r q)))) ((λy.(λu.(u (y q)))) ((λi.(λo.(o (i q)))) (λp.(p ((λa.(a (e q))) q))))))) (λs.w)) (λd.d))))
5: (λi.(λo.(((λp.((λa.(λs.(s (a i)))) ((λq.(λw.(w (q i)))) ((λe.(λr.(r (e i)))) (λt.(t (i (p i)))))))) (λy.o)) (λu.u))))
6: (λq.(λw.(((λe.((λr.(λt.(t (r q)))) ((λy.(λu.(u (y q)))) (λi.(i ((λo.(o (q (e q)))) q)))))) (λp.w)) (λa.a))))
7: (λq.(λw.(((λe.((λr.(λt.(t (r q)))) ((λy.(λu.(u (y q)))) (λi.(i (q (q (e q)))))))) (λo.w)) (λp.p))))
8: (λy.(λu.(((λi.((λo.(λq.(q (o y)))) (λw.(w ((λe.(e (y (y (i y)))))

# <span style="color: #A1F99C">6.</span>
<span style="color:#bb91f3">(PLUS 0 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [9]:
def term_test_6():
    return multi_app_term(plus_term(), num_term(0), num_term(0))


test_procedure(term_test_6())

0: (((λq.(λw.((w (λe.(λr.(λt.(r ((e r) t)))))) q))) (λy.(λu.u))) (λi.(λo.o)))
1: ((λe.((e (λr.(λt.(λy.(t ((r t) y)))))) (λu.(λi.i)))) (λq.(λw.w)))
2: (((λq.(λw.w)) (λe.(λr.(λt.(r ((e r) t)))))) (λy.(λu.u)))
3: ((λq.q) (λw.(λe.e)))
4: (λq.(λw.w))


reduction steps: 4
norm term:       (λq.(λw.w))


# <span style="color: #A1F99C">7.</span>
<span style="color:#bb91f3">(PLUS 2 3)</span> == <span style="color:#f1bd47">5</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x (x y)))))</span>

In [10]:
def term_test_7():
    return multi_app_term(plus_term(), num_term(2), num_term(3))


test_procedure(term_test_7())

0: (((λq.(λw.((w (λe.(λr.(λt.(r ((e r) t)))))) q))) (λy.(λu.(y (y u))))) (λi.(λo.(i (i (i o))))))
1: ((λq.((q (λw.(λe.(λr.(e ((w e) r)))))) (λt.(λy.(t (t y)))))) (λu.(λi.(u (u (u i))))))
2: (((λq.(λw.(q (q (q w))))) (λe.(λr.(λt.(r ((e r) t)))))) (λy.(λu.(y (y u)))))
3: ((λq.((λw.(λe.(λr.(e ((w e) r))))) ((λt.(λy.(λu.(y ((t y) u))))) ((λi.(λo.(λp.(o ((i o) p))))) q)))) (λa.(λs.(a (a s)))))
4: ((λu.((λi.(λo.(λp.(o ((i o) p))))) ((λa.(λq.(λw.(q ((a q) w))))) (λe.(λr.(e ((u e) r))))))) (λt.(λy.(t (t y)))))
5: ((λq.((λw.(λe.(λr.(e ((w e) r))))) (λt.(λy.(t (((λu.(λi.(u ((q u) i)))) t) y)))))) (λo.(λp.(o (o p)))))
6: ((λq.((λw.(λe.(λr.(e ((w e) r))))) (λt.(λy.(t ((λu.(t ((q t) u))) y)))))) (λi.(λo.(i (i o)))))
7: ((λw.((λe.(λr.(λt.(r ((e r) t))))) (λy.(λu.(y (y ((w y) u))))))) (λi.(λq.(i (i q)))))
8: ((λq.(λw.(λe.(w (((λr.(λt.(r (r ((q r) t))))) w) e))))) (λy.(λu.(y (y u)))))
9: ((λq.(λw.(λe.(w ((λr.(w (w ((q w) r)))) e))))) (λt.(λy.(t (t y)))))
10: ((λq.(λw.(λe.(w (w (w ((q w) e))))))) (λr.(

# <span style="color: #A1F99C">8.</span>
<span style="color:#bb91f3">(SUBTRACT 4 1)</span> == <span style="color:#f1bd47">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [11]:
def term_test_8():
    return multi_app_term(subtract_term(), num_term(4), num_term(1))


test_procedure(term_test_8())

0: (((λq.(λw.((w (λe.(λr.(λt.(((e (λy.(λu.(u (y r))))) (λi.t)) (λo.o)))))) q))) (λp.(λa.(p (p (p (p a))))))) (λs.(λd.(s d))))
1: ((λw.((w (λe.(λr.(λt.(((e (λy.(λu.(u (y r))))) (λi.t)) (λo.o)))))) (λp.(λa.(p (p (p (p a)))))))) (λs.(λq.(s q))))
2: (((λq.(λw.(q w))) (λe.(λr.(λt.(((e (λy.(λu.(u (y r))))) (λi.t)) (λo.o)))))) (λp.(λa.(p (p (p (p a)))))))
3: ((λq.((λw.(λe.(λr.(((w (λt.(λy.(y (t e))))) (λu.r)) (λi.i))))) q)) (λo.(λp.(o (o (o (o p)))))))
4: ((λq.(λw.(λe.(((q (λr.(λt.(t (r w))))) (λy.e)) (λu.u))))) (λi.(λo.(i (i (i (i o)))))))
5: (λi.(λq.((((λw.(λe.(w (w (w (w e)))))) (λr.(λt.(t (r i))))) (λy.q)) (λu.u))))
6: (λq.(λw.(((λe.((λr.(λt.(t (r q)))) ((λy.(λu.(u (y q)))) ((λi.(λo.(o (i q)))) ((λp.(λa.(a (p q)))) e))))) (λs.w)) (λd.d))))
7: (λq.(λw.(((λe.((λr.(λt.(t (r q)))) ((λy.(λu.(u (y q)))) ((λi.(λo.(o (i q)))) (λp.(p (e q))))))) (λa.w)) (λs.s))))
8: (λq.(λw.(((λe.((λr.(λt.(t (r q)))) ((λy.(λu.(u (y q)))) (λi.(i ((λo.(o (e q))) q)))))) (λp.w)) (λa.a))))
9: (λq.(λw.(((λe.((λr.(λt.(t

# <span style="color: #A1F99C">9.</span>
<span style="color:#bb91f3">(SUBTRACT 3 5)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [12]:
def term_test_9():
    return multi_app_term(subtract_term(), num_term(3), num_term(5))


test_procedure(term_test_9())

0: (((λq.(λw.((w (λe.(λr.(λt.(((e (λy.(λu.(u (y r))))) (λi.t)) (λo.o)))))) q))) (λp.(λa.(p (p (p a)))))) (λs.(λd.(s (s (s (s (s d))))))))
1: ((λu.((u (λi.(λo.(λp.(((i (λa.(λs.(s (a o))))) (λq.p)) (λw.w)))))) (λe.(λr.(e (e (e r))))))) (λt.(λy.(t (t (t (t (t y))))))))
2: (((λq.(λw.(q (q (q (q (q w))))))) (λe.(λr.(λt.(((e (λy.(λu.(u (y r))))) (λi.t)) (λo.o)))))) (λp.(λa.(p (p (p a))))))
3: ((λb.((λn.(λm.(λq_1.(((n (λw_1.(λe_1.(e_1 (w_1 m))))) (λr_1.q_1)) (λt_1.t_1))))) ((λy_1.(λu_1.(λi_1.(((y_1 (λo_1.(λp_1.(p_1 (o_1 u_1))))) (λa_1.i_1)) (λs_1.s_1))))) ((λq.(λw.(λe.(((q (λr.(λt.(t (r w))))) (λy.e)) (λu.u))))) ((λi.(λo.(λp.(((i (λa.(λs.(s (a o))))) (λd.p)) (λf.f))))) ((λg.(λh.(λj.(((g (λk.(λl.(l (k h))))) (λz.j)) (λx.x))))) b)))))) (λc.(λv.(c (c (c v))))))
4: ((λq.((λw.(λe.(λr.(((w (λt.(λy.(y (t e))))) (λu.r)) (λi.i))))) ((λo.(λp.(λa.(((o (λs.(λd.(d (s p))))) (λf.a)) (λg.g))))) ((λh.(λj.(λk.(((h (λl.(λz.(z (l j))))) (λx.k)) (λc.c))))) ((λv.(λb.(λn.(((v (λm.(λq_1.(q_1 (m b))))) (λw_1.n)) (λe

# <span style="color: #A1F99C">10.</span>
<span style="color:#bb91f3">(LEQ 3 2)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [13]:
def term_test_10():
    return multi_app_term(leq_term(), num_term(3), num_term(2))


test_procedure(term_test_10())

0: (((λq.(λw.((λe.((e (λr.(λt.(λy.y)))) (λu.(λi.u)))) (((λo.(λp.((p (λa.(λs.(λd.(((a (λf.(λg.(g (f s))))) (λh.d)) (λj.j)))))) o))) q) w)))) (λk.(λl.(k (k (k l)))))) (λz.(λx.(z (z x)))))
1: (((λh.(λj.((λk.((k (λl.(λz.(λq.q)))) (λw.(λe.w)))) ((λr.((r (λt.(λy.(λu.(((t (λi.(λo.(o (i y))))) (λp.u)) (λa.a)))))) h)) j)))) (λs.(λd.(s (s (s d)))))) (λf.(λg.(f (f g)))))
2: (((λq.(λw.((λe.((e (λr.(λt.(λy.y)))) (λu.(λi.u)))) ((w (λo.(λp.(λa.(((o (λs.(λd.(d (s p))))) (λf.a)) (λg.g)))))) q)))) (λh.(λj.(h (h (h j)))))) (λk.(λl.(k (k l)))))
3: (((λq.(λw.((((w (λe.(λr.(λt.(((e (λy.(λu.(u (y r))))) (λi.t)) (λo.o)))))) q) (λp.(λa.(λs.s)))) (λd.(λf.d))))) (λg.(λh.(g (g (g h)))))) (λj.(λk.(j (j k)))))
4: ((λy.((((y (λu.(λi.(λo.(((u (λp.(λa.(a (p i))))) (λs.o)) (λd.d)))))) (λf.(λg.(f (f (f g)))))) (λh.(λj.(λq.q)))) (λw.(λe.w)))) (λr.(λt.(r (r t)))))
5: (((((λq.(λw.(q (q w)))) (λe.(λr.(λt.(((e (λy.(λu.(u (y r))))) (λi.t)) (λo.o)))))) (λp.(λa.(p (p (p a)))))) (λs.(λd.(λf.f)))) (λg.(λh.g)))
6: ((((λq.((λw.(λe.

# <span style="color: #A1F99C">11.</span>
<span style="color:#bb91f3">(LEQ 2 5)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [14]:
def term_test_11():
    return multi_app_term(leq_term(), num_term(2), num_term(5))


test_procedure(term_test_11())

0: (((λq.(λw.((λe.((e (λr.(λt.(λy.y)))) (λu.(λi.u)))) (((λo.(λp.((p (λa.(λs.(λd.(((a (λf.(λg.(g (f s))))) (λh.d)) (λj.j)))))) o))) q) w)))) (λk.(λl.(k (k l))))) (λz.(λx.(z (z (z (z (z x))))))))
1: (((λj.(λk.((λl.((l (λz.(λq.(λw.w)))) (λe.(λr.e)))) ((λt.((t (λy.(λu.(λi.(((y (λo.(λp.(p (o u))))) (λa.i)) (λs.s)))))) j)) k)))) (λd.(λf.(d (d f))))) (λg.(λh.(g (g (g (g (g h))))))))
2: (((λq.(λw.((λe.((e (λr.(λt.(λy.y)))) (λu.(λi.u)))) ((w (λo.(λp.(λa.(((o (λs.(λd.(d (s p))))) (λf.a)) (λg.g)))))) q)))) (λh.(λj.(h (h j))))) (λk.(λl.(k (k (k (k (k l))))))))
3: (((λq.(λw.((((w (λe.(λr.(λt.(((e (λy.(λu.(u (y r))))) (λi.t)) (λo.o)))))) q) (λp.(λa.(λs.s)))) (λd.(λf.d))))) (λg.(λh.(g (g h))))) (λj.(λk.(j (j (j (j (j k))))))))
4: ((λu.((((u (λi.(λo.(λp.(((i (λa.(λs.(s (a o))))) (λd.p)) (λf.f)))))) (λg.(λh.(g (g h))))) (λj.(λq.(λw.w)))) (λe.(λr.e)))) (λt.(λy.(t (t (t (t (t y))))))))
5: (((((λq.(λw.(q (q (q (q (q w))))))) (λe.(λr.(λt.(((e (λy.(λu.(u (y r))))) (λi.t)) (λo.o)))))) (λp.(λa.(p (p a))))) (λ

# <span style="color: #A1F99C">12.</span>
<span style="color:#bb91f3">(EQ 2 2)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [17]:
def term_test_12():
    return multi_app_term(eq_term(), num_term(2), num_term(2))


test_procedure(term_test_12(), steps_lim=100)

0: (((λa_1.(λs_1.(((λd_1.(λf_1.((((λg_1.(λh_1.(λj_1.((g_1 h_1) j_1)))) d_1) f_1) d_1))) (((λk_1.(λl_1.((λq.((q (λw.(λe.(λr.r)))) (λt.(λy.t)))) (((λu.(λi.((i (λo.(λp.(λa.(((o (λs.(λd.(d (s p))))) (λf.a)) (λg.g)))))) u))) k_1) l_1)))) a_1) s_1)) (((λh.(λj.((λk.((k (λl.(λz.(λx.x)))) (λc.(λv.c)))) (((λb.(λn.((n (λm.(λq_1.(λw_1.(((m (λe_1.(λr_1.(r_1 (e_1 q_1))))) (λt_1.w_1)) (λy_1.y_1)))))) b))) h) j)))) s_1) a_1)))) (λu_1.(λi_1.(u_1 (u_1 i_1))))) (λo_1.(λp_1.(o_1 (o_1 p_1)))))
1: (((λq.(λw.(((λe.(λr.((((λt.(λy.(λu.((t y) u)))) e) r) e))) (((λi.(λo.((λp.((p (λa.(λs.(λd.d)))) (λf.(λg.f)))) (((λh.(λj.((j (λk.(λl.(λz.(((k (λx.(λc.(c (x l))))) (λv.z)) (λb.b)))))) h))) i) o)))) q) w)) (((λn.(λm.((λq_1.((q_1 (λw_1.(λe_1.(λr_1.r_1)))) (λt_1.(λy_1.t_1)))) ((λu_1.((u_1 (λi_1.(λo_1.(λp_1.(((i_1 (λa_1.(λs_1.(s_1 (a_1 o_1))))) (λd_1.p_1)) (λf_1.f_1)))))) n)) m)))) w) q)))) (λg_1.(λh_1.(g_1 (g_1 h_1))))) (λj_1.(λk_1.(j_1 (j_1 k_1)))))
2: (((λq.(λw.(((λe.(λr.((((λt.(λy.(λu.((t y) u)))) e) r) e))) (((λi.(

# <span style="color: #A1F99C">13.</span>
<span style="color:#bb91f3">(EQ 1 4)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [18]:
def term_test_13():
    return multi_app_term(eq_term(), num_term(1), num_term(4))


test_procedure(term_test_13(), steps_lim=100)

0: (((λq.(λw.(((λe.(λr.((((λt.(λy.(λu.((t y) u)))) e) r) e))) (((λi.(λo.((λp.((p (λa.(λs.(λd.d)))) (λf.(λg.f)))) (((λh.(λj.((j (λk.(λl.(λz.(((k (λx.(λc.(c (x l))))) (λv.z)) (λb.b)))))) h))) i) o)))) q) w)) (((λn.(λm.((λq_1.((q_1 (λw_1.(λe_1.(λr_1.r_1)))) (λt_1.(λy_1.t_1)))) (((λu_1.(λi_1.((i_1 (λo_1.(λp_1.(λa_1.(((o_1 (λs_1.(λd_1.(d_1 (s_1 p_1))))) (λf_1.a_1)) (λg_1.g_1)))))) u_1))) n) m)))) w) q)))) (λh_1.(λj_1.(h_1 j_1)))) (λk_1.(λl_1.(k_1 (k_1 (k_1 (k_1 l_1)))))))
1: (((λp_1.(λa_1.(((λs_1.(λd_1.((((λf_1.(λg_1.(λh_1.((f_1 g_1) h_1)))) s_1) d_1) s_1))) (((λj_1.(λk_1.((λq.((q (λw.(λe.(λr.r)))) (λt.(λy.t)))) (((λu.(λi.((i (λo.(λp.(λa.(((o (λs.(λd.(d (s p))))) (λf.a)) (λg.g)))))) u))) j_1) k_1)))) p_1) a_1)) (((λh.(λj.((λk.((k (λl.(λz.(λx.x)))) (λc.(λv.c)))) ((λb.((b (λn.(λm.(λq_1.(((n (λw_1.(λe_1.(e_1 (w_1 m))))) (λr_1.q_1)) (λt_1.t_1)))))) h)) j)))) a_1) p_1)))) (λy_1.(λu_1.(y_1 u_1)))) (λi_1.(λo_1.(i_1 (i_1 (i_1 (i_1 o_1)))))))
2: (((λq.(λw.(((λe.(λr.((((λt.(λy.(λu.((t y) u)))) e) r) 